# Dowenload era5 land using era5cli

This notebook shows how to download [era5 land data](https://climate.copernicus.eu/) using [era5cli](https://era5cli.readthedocs.io/en/latest/index.html) for 170 sites. More specifically, at each site:

- it gets the location of the site
- it checks the site location against ERA5 land sea mask
- it gets the starting time of forcing data 
- it donwloads 9 variables at one location and one time  

In [1]:
# import python libraries
from netCDF4 import Dataset, num2date
from pathlib import Path
import xarray as xr
import numpy as np

## Settings

Run these the cell below only once for:

1- setting cds keys to access data

2- installing era5cli to download data

In [ ]:
# Create a key ascii file
!echo "url: https://cds.climate.copernicus.eu/api/v2" > ~/.cdsapirc 

# Replace UID with your user ID and KEY with your API key
!echo "key: UID:KEY" >> ~/.cdsapirc

# install era5cli
!pip install era5cli

## Path to land sea mask and forcing data

ERA5-land provides `land_sea mask`. If you click on [land_sea mask](https://confluence.ecmwf.int/download/attachments/140385202/lsm_1279l4_0.1x0.1.grb_v4_unpack.nc?version=1&modificationDate=1591979822208&api=v2), a netcdf file will be donwloaded. See [documentation](https://confluence.ecmwf.int/display/FUG/Land-Sea+Mask). 
Download `land_sea mask` file and set the path to the file.

In [2]:
# Path to land_sea mask file
land_sea_filename = Path("./lsm_1279l4_0.1x0.1.grb_v4_unpack.nc")

In [3]:
# path to forcing data, for example on CRIB data are located at:
forcing_path = Path("/data/shared/EcoExtreML/STEMMUS_SCOPEv1.0.0/input/Plumber2_data/")

## Functions to select data in era5-land 

In [4]:
def find_nearest_ERA5_land_point(
    lsm_dataset,
    point_longitude,
    point_latitude,
    padding,
):
    """Find nearest grid cell in land see mask to a point based on Geographical distances.
    args:
        lsm_dataset: ERA5 land sea mask dataset
        point_longitude: longitude in degrees of target coordinate
        point_latitude: latitude in degrees of target coordinate
        padding: bounding box around the point
    returns:
        longitude: longitude of closest grid cell
        latitude: latitude of closest grid cell
    """
    # create a bounding_box 
    lat_max = point_latitude + padding
    lat_min = point_latitude - padding
    lon_max = point_longitude + padding
    lon_min = point_longitude - padding

    lsm_lon = lsm_dataset["longitude"].values
    lsm_lat = lsm_dataset["latitude"].values
    longitudes = lsm_lon[(lsm_lon >= lon_min) & (lsm_lon <= lon_max)]
    latitudes = lsm_lat[(lsm_lat >= lat_min) & (lsm_lat <= lat_max)]
    bounding_box = lsm_dataset.sel(longitude=longitudes, latitude=latitudes)

    grid_longitudes_array = np.array(bounding_box["longitude"].values)
    grid_latitudes_array = np.array(bounding_box["latitude"].values)

    # Create a grid from coordinates (shape will be (nlat, nlon))
    lon_vectors, lat_vectors = np.meshgrid(grid_longitudes_array, grid_latitudes_array)

    # calculate distanced
    distances = geographical_distances(
        point_longitude, point_latitude, lon_vectors, lat_vectors
    )

    # mask distances where they are on sea: values less than 0.6
    masked_distances = xr.where(bounding_box >= 0.6, distances, np.nan)

    # find the nearest on land
    index = np.nanargmin(masked_distances["lsm"].to_numpy())
    idx_lat, idx_lon = np.unravel_index(index, distances.shape)
    
    selected_grid = bounding_box.isel(longitude=int(idx_lon), latitude=int(idx_lat))
    
    return selected_grid["longitude"].values, selected_grid["latitude"].values


def geographical_distances(
    point_longitude,
    point_latitude,
    lon_vectors,
    lat_vectors,
    radius=6373.0,
):
    """It uses Spherical Earth projected to a plane formula:
    https://en.wikipedia.org/wiki/Geographical_distance
    args:
        point_longitude: longitude in degrees of target coordinate
        point_latitude: latitude in degrees of target coordinate
        lon_vectors: 1d array of longitudes in degrees
        lat_vectors: 1d array of latitudes in degrees
        radius: Radius of a sphere in km. Default is Earths approximate radius.
    returns:
        distances: array of geographical distance of point to all vector members
    """
    dlon = np.radians(lon_vectors - point_longitude)
    dlat = np.radians(lat_vectors - point_latitude)
    latm = np.radians((lat_vectors + point_latitude) / 2)
    return radius * np.sqrt(dlat**2 + (np.cos(latm) * dlon) ** 2)


def convert_lon(ds, lon_name):
    """Adjust lon values to make sure they are within (-180, 180).
    args: 
        ds: xarray dataset
        lon_name: whatever name is in the data
    returns: 
        ds: xarray dataset
    """
    ds['_longitude_adjusted'] = xr.where(
        ds[lon_name] > 180,
        ds[lon_name] - 360,
        ds[lon_name])

    # reassign the new coords to as the main lon coords
    # and sort DataArray using new coordinate values
    ds = (
        ds
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(ds._longitude_adjusted)})
        .drop(lon_name))

    ds = ds.rename({'_longitude_adjusted': lon_name})
    return ds


def get_forcing_info(ds):
    
    lon = forcing["longitude"].values.flatten()[0]
    lat = forcing["latitude"].values.flatten()[0]
    time = forcing["time"].dt
    return lon, lat, time
    

## Get site information and check it against land sea mask

In [5]:
# Open land sea mask data and convert the longitude values
# In forcing data lon (-180, 180) and lat (-90, 90)
# In land sea mask lon (0, 360) and lat (-90, 90)
lsm = xr.open_dataset(land_sea_filename)
lsm_dataset = convert_lon(lsm, 'longitude')

In [17]:
# Choose a site and get forcing information like location and strating time
forcing_name = "ES-ES1_1999-2006_LaThuile_Met.nc"
forcing = xr.open_dataset(forcing_path / forcing_name)
lon, lat, time =  get_forcing_info(forcing)
print(lon, lat)

-0.31881 39.34597


In [7]:
# check the site location againts land sea mask 
nearest_lon, nearest_lat = find_nearest_ERA5_land_point(lsm_dataset, lon, lat, 0.2)
print(nearest_lon, nearest_lat)

-0.3999939 39.3


## Prepare required information for era5cli command line

In [18]:
# starting time
startyear = time.year.values[0]
endyear = startyear
startmonth = time.month.values[0]
startday = time.day.values[0]
starthour = time.hour.values[0]

# create area, note that era5 resolution is 0.1
# and we want only one grid cell
lat_max = nearest_lat + 0.001
lat_min = nearest_lat - 0.001
lon_max = nearest_lon + 0.001
lon_min = nearest_lon - 0.001

# create prefix for file name
station_name = forcing_name.split('_')[0]
timestamp = time.strftime('%Y%m%d_%H').values[0]
file_name = f"{station_name}_{timestamp}"

## Test the download command line using --dryrun
The cell below makes sure that the command line works correctly. It only shows what ERA5cli command-line will return. It does **not** download any data. 

In [16]:
# Run era5cli for variables soil_temperature and volumetric_soil_water at four levels and skin_temperature
!era5cli hourly --variables soil_temperature_level_1 soil_temperature_level_2 soil_temperature_level_3 soil_temperature_level_4 \
volumetric_soil_water_layer_1 volumetric_soil_water_layer_2 volumetric_soil_water_layer_3 volumetric_soil_water_layer_4 skin_temperature \
--startyear {startyear} --endyear {endyear} --land \
--area {lat_max} {lon_min} {lat_min} {lon_max} \
--months {startmonth} --days {startday} --hours {starthour} --outputprefix {file_name} --dryrun --merge

NB: coordinates [39.300999237060545, -0.400993896484375, 39.29899923706055, -0.398993896484375] rounded down to two decimals.

NB: coordinates [39.300999237060545, -0.400993896484375, 39.29899923706055, -0.398993896484375] rounded down to two decimals.

NB: coordinates [39.300999237060545, -0.400993896484375, 39.29899923706055, -0.398993896484375] rounded down to two decimals.

NB: coordinates [39.300999237060545, -0.400993896484375, 39.29899923706055, -0.398993896484375] rounded down to two decimals.

NB: coordinates [39.300999237060545, -0.400993896484375, 39.29899923706055, -0.398993896484375] rounded down to two decimals.
reanalysis-era5-land {'variable': 'soil_temperature_level_1', 'year': [1999], 'month': ['01'], 'time': ['00:00'], 'format': 'netcdf', 'area': [39.3, -0.4, 39.3, -0.4], 'day': ['01']} ES-ES1_19990101_00-land_soil_temperature_level_1_1999_hourly_0E-0E_39N-39N.nc
reanalysis-era5-land {'variable': 'volumetric_soil_water_layer_2', 'year': [1999], 'month': ['01'], 'time

## Download data

The cell below downloads data. You can specify the name of forcing files or download data at all 170 sites. 

In [ ]:
## specify the forcing file names or set fullrun = True
forcing_filenames_list = ["AR-SLu_2010-2010_FLUXNET2015_Met.nc", "AU-ASM_2011-2017_OzFlux_Met.nc"]

## if you want to download all 170 sites, change fullrun = False to fullrun = True
fullrun = False
if fullrun:
    forcing_filenames_list = [file.name for file in Path(forcing_path).iterdir()]
    
for forcing_name in forcing_filenames_list:
    # read data and get time and coordinates
    forcing = xr.open_dataset(forcing_path / forcing_name)
    lon, lat, time =  get_forcing_info(forcing)
    
    # check site location on land sea mask
    nearest_lon, nearest_lat = find_nearest_ERA5_land_point(lsm_dataset, lon, lat, 0.2)
    
    # get starting time
    startyear = time.year.values[0]
    endyear = startyear
    startmonth = time.month.values[0]
    startday = time.day.values[0]
    starthour = time.hour.values[0]

    # create area, note that era5 resolution is 0.1
    # and we want only one grid cell
    lat_max = nearest_lat + 0.001
    lat_min = nearest_lat - 0.001
    lon_max = nearest_lon + 0.001
    lon_min = nearest_lon - 0.001

    # create prefix for file name
    station_name = forcing_name.split('_')[0]
    timestamp = time.strftime('%Y%m%d_%H').values[0]
    file_name = f"{station_name}_{timestamp}"

    # download data
    !era5cli hourly --variables soil_temperature_level_1 soil_temperature_level_2 soil_temperature_level_3 soil_temperature_level_4 \
    volumetric_soil_water_layer_1 volumetric_soil_water_layer_2 volumetric_soil_water_layer_3 volumetric_soil_water_layer_4 skin_temperature \
    --startyear {startyear} --endyear {endyear} --land \
    --area {lat_max} {lon_min} {lat_min} {lon_max} \
    --months {startmonth} --days {startday} --hours {starthour} --outputprefix {file_name} --merge